# PC 4 : Apprentissage supervisé et prétraitement - 23 juin 2025 - Solution

Dans ce notebook, nous allons explorer quelques méthodes de prétraitement des données et leur impact sur une régression linéaire. 

Ce notebook vous permettra ainsi de découvrir des fonctionalités de `scikit-learn` permettant :
* d'entrainer et évaluer un algorithme d'apprentissage supervisé
* d'encoder des variables qualitatives ;
* de ramener des variables à une fourchette de valeurs ;
* de transformer des variables pour rapprocher leur distribution de celle d'une gaussienne.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Données
Dans ce notebook nous allons travailler avec les données contenues dans le fichier `../../data/auto-mpg.tsv`. Ces données, obtenues sur https://archive.ics.uci.edu/ml/datasets/Auto+MPG, décrivent des voitures par les variables suivantes :

    1. mpg:           consommation (en miles par gallon), continue
    2. cylinders:     nombre de cylindres, discrète
    3. displacement:  cylindrée, continue
    4. horsepower:    chevaux-vapeur, continue
    5. weight:        poids, continue
    6. acceleration:  accélération, continue
    7. model year:    année, discrète
    8. origin:        région d'origine, discrète (1=Amérique du Nord ; 2=Europe ; 3=Asie)
    9. car name:      nom, chaîne de caractères.

Notre but va être de prédire la consommation d'un véhicule à partir des autres variables (à l'exclusion du nom de la voiture, qui est un identifiant unique).

In [ ]:
# Charger les données
df = pd.read_csv("data/auto-mpg.tsv", delimiter='\t')

In [ ]:
df.head()

### Création des matrices X et y de données

In [ ]:
X = np.array(df.drop(columns=['mpg', 'car name']))

In [ ]:
y = np.array(df['mpg'])

### Séparation en un jeu d'entraînement et un jeu de test

Notre but étant de construire un modèle prédictif, nous allons pour l'évaluer mettre de côté une partie des données (20%), le jeu de test, que nous n'utiliserons pas pour l'entraînement. Rappelez-vous que la minimisation du risque empirique ne garantit pas la minimisation du risque : la performance d'un modèle sur les données sur lesquelles il est entraîné peut être excellente, sans que celui-ci fasse de bonnes prédictions sur d'autres individus. Vous pouvez comparer ça à apprendre par cœur le jeu d'entraînement. Nous parlerons plus en détail de sélection et évaluation de modèle au chapitre 8. 

L'utilisation de l'argument `random_state` garantit que la répartition des individus entre les deux jeux soit toujours la même au sein de ce notebook si vous relancez la commande. Attention, vous n'aurez a priori pas la même que quelqu'un d'autre exécutant le même notebook sur une autre machine, ce qui peut expliquer de fluctuations entre vos résultats et ceux d'une autre personne.

In [ ]:
from sklearn import model_selection

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=0.20, 
                                                                    random_state=27)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

__Question :__ Combien d'observations et de variables contiennent, respectivement, le jeu d'échantillon et le jeu de test ?

__Réponse :__ Le jeu d'entraînement contient 313 échantillons. Le jeu de test en contient 79. Les données sont représentées par 7 variables.

## 2. Visualisation des données

Nous allons maintenant visualiser les variables représentant nos véhicules. Pour ce faire, nous allons séparer les variables continues (que nous allons représenter chacune par un histogramme) des variables discrètes (que nous allons représenter chacune par un diagramme en barre).

Nous nous concentrons sur le jeu d'entraînement : notre but est d'utiliser le jeu de test pour tester les modèles appris sur le jeu d'entraînement, en prétendant ne pas le connaître au moment de l'entraînement.

N'hésitez pas à ajuster les paramètres des méthodes de `matplotlib` pour produire des graphiques plus lisibles.

In [ ]:
continuous_features = ['displacement', 'horsepower', 'weight', 'acceleration']
discrete_features = ['cylinders', 'model year', 'origin']

features = list(df.drop(columns=['mpg', 'car name']).columns)

continuous_features_idx = [features.index(feat_name) for feat_name in continuous_features]
discrete_features_idx = [features.index(feat_name) for feat_name in discrete_features]

Nous allons maintenant représenter les variables discrètes par des diagrammes en barres :

In [ ]:
# Diagramme en barres pour les variables discrètes
fig = plt.figure(figsize=(12, 3))

for (plot_idx, feat_idx) in enumerate(discrete_features_idx):
    # créer une sous-figure (subplot) à la position (plot_idx+1) d'une grille 1x3
    ax = fig.add_subplot(1, 3, (plot_idx+1))

    # calculer la fréquence de chacune des valeurs prise par la variable feat_idx
    feature_values = np.unique(X_train[:, feat_idx])
    frequencies = [(float(len(np.where(X_train[:, feat_idx]==value)[0]))/X_train.shape[0]) \
                   for value in feature_values]

    # afficher le diagramme en barres pour la variable feat_idx
    b = ax.bar(range(len(feature_values)), frequencies, width=0.5, 
               tick_label=list([int(n) for n in feature_values]))
    
    # utiliser le nom de la variable comme titre pour chaque histogramme
    ax.set_title(features[feat_idx])
fig.tight_layout(pad=1.0)

__Question :__ En vous inspirant du code ci-dessus et de la PC3, affichez les histogrammes des variables continues.

In [ ]:
# Réponse :
fig = plt.figure(figsize=(5, 5))

# Histogrammes pour les variables continues
for (plot_idx, feat_idx) in enumerate(continuous_features_idx):
    # créer une sous-figure (subplot) à la position (plot_idx+1) d'une grille 2x2
    ax = fig.add_subplot(2, 2, (plot_idx+1))
    
    # afficher l'histogramme de la variable feat_idx
    h = ax.hist(X_train[:, feat_idx], bins=30, edgecolor='none')
    
    # utiliser le nom de la variable comme titre pour chaque diagramme
    ax.set_title(features[feat_idx])
    
# espacement entre les subplots
fig.tight_layout(pad=1.0)

__Question :__ Quelles sont les fourchettes de valeur prises par les différentes variables ?

__Réponse :__ Il s'agit ici d'observer que certaines valeurs ont des ordres de grandeur différents d'autres (poids vs accélération, année ou pays).

__Question :__ Tracer l'histogramme des étiquettes du jeu d'entrainement.

In [ ]:
plt.hist(y_train, bins=30, edgecolor='none')
plt.title('mpg')

## 3. Un premier modèle

Nous allons maintenant construire une _baseline_, c'est-à-dire un premier modèle qui nous servira de point de comparaison. Ici, il s'agit d'utiliser `scikit-learn` pour entraîner une régression linéaire sur les données sans les prétraiter.

Les modèles linéaires de `scikit-learn` sont implémentés dans le module [`sklearn.linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model). __N'hésitez pas à vous référer fréquemment à la documentation de scikit-learn, qui est très complète.__

In [ ]:
from sklearn import linear_model

### Entraînement du modèle

In [ ]:
# Instanciation d'un objet LinearRegression
predictor = linear_model.LinearRegression()

In [ ]:
# Entrainement de cet objet sur les données d'entraînement
predictor.fit(X_train, y_train)

### Prédiction sur les données de test

In [ ]:
y_pred = predictor.predict(X_test)

### Performance

Il s'agit maintenant d'évaluer ces prédictions. Pour cela, nous allons utiliser les fonctionalités du module [`metrics`](https://scikit-learn.org/stable/api/sklearn.metrics.html) de `scikit-learn`.

Comme il s'agit d'un problème de régression, nous allons utiliser la __RMSE__ (_Root Mean Squared Error_) comme mesure de la performance du modèle : il s'agit de la racine carrée de la moyenne des carrés des erreurs.

__Question :__ Pourquoi prendre la racine carrée et pas simplement la MSE ?

__Réponse :__ Pour des questions d'homogénéité : la RMSE s'exprime dans la même unité que l'étiquette.

In [ ]:
from sklearn import metrics

In [ ]:
print(f"RMSE: {metrics.root_mean_squared_error(y_test, y_pred):.2f}")

__Question :__ Que pensez-vous de cette erreur ? Est-elle faible? Grande ?

__Réponse :__ Vu la fourchette de valeurs des étiquettes (environ entre 10 et 45), une erreur de 3.29 n'est pas si mal.

Nous pouvons aussi utiliser une visualisation, et représenter chaque individu du jeu de test par son étiquette prédite vs. sa vraie étiquette.

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_test, y_pred)

plt.xlabel("Consommation réelle (mpg)")
plt.ylabel("Consommation prédite (mpg)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y_test), np.min(y_pred)])-1
axis_max = np.max([np.max(y_test), np.max(y_pred)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

### Coefficients de régression

Pour comprendre notre modèle, nous pouvons regarder les coefficients affectés à chaque variable dans le modèle linéaire appris.

In [ ]:
# Afficher, pour chaque variable, son poids dans le modèle (en valeur absolue)
num_features = X_train.shape[1]
feature_names = df.drop(columns=['mpg', 'car name']).columns
plt.scatter(range(num_features), np.abs(predictor.coef_))

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features), feature_names, rotation=90)
tmp = plt.ylabel('Coefficient')

__Question :__ Quelle variable a le plus fort coefficient (en valeur absolue) ? Pensez-vous que cela signifie que cette variable joue un rôle très important dans la prédiction ?

__Réponse :__ C'est l'origine géographique de la voiture. Il ne faut pas oublier que les variables prennent des valeurs très différentes, `origin` a des valeurs entre 1 et 3 et `weight` entre 2000 et 5000... L'interprétation est difficile.

### Corrélation

La corrélation des variables peut brouiller notre interprétation des poids attribués à chaque variable par notre modèle. 

**Question :** Représentez la corrélation entre les variables de notre jeu de données (s'inspirer de la PC3. Attention, la corrélation n'a de sens qu'entre des variables quantitatives). Qu'en conclure quant à l'interprétabilité des coefficients affectés aux variables ?

In [ ]:
import seaborn as sns

corr_mat = df.drop(columns=['mpg', 'car name', 'origin']).corr() # enlever origin dont les valeurs numériques ne sont qu'un encodage de régions du monde.
plt.figure(figsize=(5, 4))
# Affichage heatmap
sns.heatmap(corr_mat, 
            vmin=-1, # borne inf des valeurs à afficher
            vmax=1, # borne sup des valeurs à afficher
            center= 0, # valeur médiane des valeurs à afficher,
            cmap='PuOr', # colormap divergente de violet (PUrple) vers orange (ORange)
           )

Les quatres variables cylinders, displacement, horspower et weight sont fortement positivement corrélées. Plusieurs solutions pour les valeurs de ces coefficients sont possibles, en "compensant" un changement dans un coefficient par d'autres changements dans les coefficients des autres variables corrélées. Il est difficile d'interpréter les valeurs des coefficients dans ces conditions.

## 4. Encodage des variables qualitatives
La variable `origin` est une variable qualitative : l'encodage 1-2-3 est tout à fait arbitraire. Il suppose en particulier, si on réfléchit en termes de distances, que l'Asie est deux fois plus loin de l'Amérique du Nord que de l'Europe, ce qui n'a aucun sens.

Un encodage plus raisonnable pour ce genre de cas est ce qu'on appelle l'encodage _one-hot_, ou encore _dummy encoding_ : on représente la variable par autant de variables binaires qu'il y a de valeurs possibles (3 dans le cas de la variable `origin` : la première correspond à Amérique du Nord, la deuxième à Europe, la troisième à Asie), et on met à `1` la seule de ces variables binaires correspondant à la valeur que l'on encode.

Ainsi l'unique variable `origin` devient 3 variables binaires:
```    
   Amérique du Nord --> 1, 0, 0
   Europe --> 0, 1, 0
   Asie --> 0, 0, 1
```  
Cette représentation a l'inconvénient d'augmenter le nombre de variables, mais les distances euclidiennes sont maintenant plus raisonnables (elles valent 1 si les valeurs sont différentes et 0 si elles sont identiques).

Cette fonctionalité existe dans `pandas` comme dans `scikit-learn`. 

In [ ]:
# Créer un nouveau data frame où la colomne 'origin' est remplacée par son encodage 'one-hot'
df_dummies = pd.get_dummies(df, columns=['origin'])

In [ ]:
df_dummies.head()

In [ ]:
# Extraire de nouveau les données
X_dummies = np.array(df_dummies.drop(columns=['mpg', 'car name']))

In [ ]:
X_dummies_train, X_dummies_test, y_d_train, y_d_test = model_selection.train_test_split(X_dummies, y, 
                                                                                        test_size=0.20, 
                                                                                        random_state=27)
                                                                                                    

Vous pouvez vérifier ici que l'utilisation de la même graine pour `random_state` génère la même répartition des données que précédemment :

In [ ]:
np.sum(np.abs(y_d_train-y_train)) + np.sum(np.abs(y_d_test-y_test))

### Impact sur le modèle

Nous allons maintenant apprendre une régression linéaire sur les données où la variable `origin` a été remplacée par son encodage one-hot. 

__Question :__ Créer une instance `predictor_dummy` de la classe `LinearRegression` entraînée sur les données contenant la version _one-hot_ de la variable `origin`.

In [ ]:
# Créer un nouvel objet LinearRegression 
predictor_dummy = linear_model.LinearRegression()

# Entraîner predictor_dummy sur les nouvelles données
predictor_dummy.fit(X_dummies_train, y_train)

__Question :__ Créer un array `y_pred_dummy` qui contient les prédictions de la nouvelle régression linéaire sur les données de test.

In [ ]:
y_pred_dummy = predictor_dummy.predict(X_dummies_test)

__Question :__ Quelle est la RMSE, sur le jeu de test, de ce nouveau modèle ? La comparer à la RMSE précédente.

In [ ]:
print(f"RMSE (encodage one-hot): {metrics.root_mean_squared_error(y_test, y_pred_dummy):.2f}")

__Réponse :__ La RMSE est un tout petit peu plus faible.

### Comparaison aux prédictions de la baseline

Les performances sont-elles vraiment différentes ? Nous pouvons comparer les prédictions directement.

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_test, y_pred, label='Baseline')
plt.scatter(y_test, y_pred_dummy, label='Avec one-hot')

plt.xlabel("Consommation réelle (mpg)")
plt.ylabel("Consommation prédite (mpg)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y_test), np.min(y_pred), np.min(y_pred_dummy)])-1
axis_max = np.max([np.max(y_test), np.max(y_pred), np.max(y_pred_dummy)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

plt.legend(loc=(0.02, 0.8))

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_pred, y_pred_dummy)

plt.xlabel("Consommation prédite (mpg) (baseline)")
plt.ylabel("Consommation prédite (mpg) (avec one-hot)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y_pred), np.min(y_pred_dummy)])-1
axis_max = np.max([np.max(y_pred), np.max(y_pred_dummy)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

In [ ]:
import scipy.stats as st

In [ ]:
r, pval = st.pearsonr(y_pred, y_pred_dummy)
print(f"Corrélation entre les prédictions : : {r:.2f} (p={pval:.2e})")

### Coefficients de régression

Comparons maintenant les deux modèles visuellement :

__Question :__ Afficher, sur le même graphique, les poids (en valeur absolue) de chaque variable dans chacun des deux modèles `predictor` et `predictor_dummy`. 

In [ ]:
# Même code que précédemment pour les poids de predictor
num_features = X_train.shape[1]
plt.scatter(range(num_features), np.abs(predictor.coef_), label='Original')

# Code adapté pour les poids de predictor_dummy
num_features2 = X_dummies_train.shape[1]
plt.scatter(range(num_features2), np.abs(predictor_dummy.coef_), label='Avec one-hot', marker='v')
feature_names2 = df_dummies.drop(columns=['mpg', 'car name']).columns

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features2), feature_names2, rotation=90)
tmp = plt.ylabel('Coefficient')

plt.legend(loc=(0.02, 0.75))

__Question :__ Ce modèle est-il vraiment différent du précédent ?

__Réponse :__ La contribution de chaque origine au modèle précédent est :
```
    Amérique du Nord : predictor.coef_[-1] ~ 1.62
    Europe : 1.62 x 2 ~ 3.24
    Asie : 1.62 x 3 ~ 4.86
```
Pour le nouveau modèle :
```
    Amérique du Nord : predictor_dummy.coef_[-3] ~ -1.95
    Europe : predictor_dummy.coef_[-2] ~ 0.68
    Asie : predictor_dummy.coef_[-1] ~ 1.28
```
Les autres coefficients ne changent pas.

Les modèles sont très légèrements différents mais cette différence a peu d'impact. __Ça ne veut pas dire que cette transformation n'a jamais d'intérêt en général...__    

## 5. Ramener les variables à des échelles comparables

Le fait que les variables soient sur des échelles différentes rend l'interprétation des coefficients de la régression linéaire délicate. 

### 5.1 Centrer et réduire les variables

Centrer et réduire les variables (comme nous l'avons vu dans la PC3) permet de remédier à ce problème.

#### Transformation des variables

In [ ]:
from sklearn import preprocessing

In [ ]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(X_train)

In [ ]:
X_train_scaled = standard_scaler.transform(X_train)
X_test_scaled = standard_scaler.transform(X_test)

#### Visualisation des nouvelles variables

__Question :__ En s'inspirant du code de la section 2, créer une grille de figures de taille 4x2, et affichez côte à côte, pour chacune des variables continues, en bleu son histogramme dans `X_train` et en orange son histogramme dans `X_train_scaled`.

In [ ]:
fig = plt.figure(figsize=(8, 10))

# Histogrammes pour les variables continues
for (plot_idx, feat_idx) in enumerate(continuous_features_idx):
    # créer une sous-figure (subplot) à la position (plot_idx+1) d'une grille 4x2
    ax = fig.add_subplot(4, 2, (2*plot_idx+1))
    
    # afficher l'histogramme de la variable feat_idx dans X_train
    h = ax.hist(X_train[:, feat_idx], bins=30, edgecolor='none')
    
    # utiliser le nom de la variable comme titre de l'histogramme
    ax.set_title("%s (original)" % features[feat_idx])    
    
    # créer une sous-figure (subplot) à la position (plot_idx+2) d'une grille 4x2
    ax = fig.add_subplot(4, 2, (2*plot_idx+2))
    
    # afficher l'histogramme de la variable feat_idx dans X_train_scaled
    h = ax.hist(X_train_scaled[:, feat_idx], bins=30, edgecolor='none', color='orange')
    
    # utiliser le nom de la variable comme titre de l'histogramme
    ax.set_title("%s (centrée-réduite)" % features[feat_idx])
    
# espacement entre les subplots
fig.tight_layout(pad=1.0)

__Question :__ En s'inspirant du code de la section 2, créer une grille de figures de taille 3x2, et affichez côte à côte, pour chacune des variables discrètes, en bleu le diagramme en barre des fréquences de ses valeurs prises dans `X_train`, et en orange le diagramme en barre des fréquences de ses valeurs prises dans `X_train_scaled`.

In [ ]:
fig = plt.figure(figsize=(10, 10))

# Variables dans X_train
for (plot_idx, feat_idx) in enumerate(discrete_features_idx):
    # créer une sous-figure (subplot) à la position (2*plot_idx+1) d'une grille 3x2
    ax = fig.add_subplot(3, 2, (2*plot_idx+1))
    
    # créeer le diagramme en barre comme précédemment
    feature_values = np.unique(X_train[:, feat_idx])
    frequencies = [(float(len(np.where(X_train[:, feat_idx]==value)[0]))/X_train.shape[0]) \
                   for value in feature_values]
    tick_labels = feature_values.astype(int)
    b = ax.bar(range(len(feature_values)), frequencies, width=0.5, tick_label=tick_labels)    
    
    # utiliser le nom de la variable comme titre de l'histogramme
    ax.set_title("%s (originale)" % features[feat_idx])

# Variables dans X_train_scaled
for (plot_idx, feat_idx) in enumerate(discrete_features_idx):
    # créer une sous-figure (subplot) à la position (2*plot_idx+2) d'une grille 3x2
    ax = fig.add_subplot(3, 2, (2*plot_idx+2))
    
    # créeer le diagramme en barre comme précédemment
    feature_values = np.unique(X_train_scaled[:, feat_idx])
    frequencies = [(float(len(np.where(X_train_scaled[:, feat_idx]==value)[0]))/X_train_scaled.shape[0]) \
                   for value in feature_values]
    tick_labels = ["%.1f" % v for v in feature_values]
    b = ax.bar(range(len(feature_values)), frequencies, width=0.5, 
               tick_label=tick_labels, color='orange')  
    
    # utiliser le nom de la variable comme titre de l'histogramme
    ax.set_title("%s (centrée-réduite)" % features[feat_idx])

# espacement entre les subplots
fig.tight_layout(pad=1.0)

#### Impact sur le modèle

__Question :__ Entrainez un modèle `predictor_scaled` sur les données centrées-réduites.

In [ ]:
# Créer un nouvel objet LinearRegression 
predictor_scaled = linear_model.LinearRegression()

# Entraîner predictor_scaled sur les nouvelles données
predictor_scaled.fit(X_train_scaled, y_train)

__Question :__ Créer un array `y_pred_scaled` qui contient les prédictions de `predictor_scaled` sur le jeu de test.

In [ ]:
y_pred_scaled = predictor_scaled.predict(X_test_scaled)

__Question :__ Quelle est la RMSE, sur le jeu de test, de ce nouveau modèle ? La comparer à la RMSE précédente.

In [ ]:
print(f"RMSE (scaled): {metrics.root_mean_squared_error(y_test, y_pred_scaled):.2f}")

__Réponse :__ La RMSE n'a pas changé.

__Question :__ Comparer les coefficients de régression des deux modèles. Quelles sont les variables les plus pertinentes pour prédire la consommation d'un véhicule ?

In [ ]:
# Comme précédemment pour les coefficients de régression dans predictor
num_features = X_train.shape[1]
plt.scatter(range(num_features), np.abs(predictor.coef_), label='Originales')

# Même code modifié pour afficher les coefficients de régression dans predictor_scaled
plt.scatter(range(num_features), np.abs(predictor_scaled.coef_), label='Centrées-réduites', marker='v')

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features), feature_names, rotation=90)
tmp = plt.ylabel('Coefficient')
plt.legend(loc=(0.02, 0.75))

__Réponse :__ On peut maintenant interpréter les coefficients de régression : les variables les plus importantes sont le poids, l'année et la cylindrée. On voit que l'origine est beaucoup moins pertinente qu'on aurait pu le croire.

__Remarque :__ Réfléchir au fait que les prédictions sont exactement les mêmes mais pas les coefficients.

### 5.2 Réduction min-max
La __réduction min-max__ est une autre façon de ramener les variables sur une même échelle, en les ramenant entre 0 et 1 par $x_j \leftarrow (x_j - \min(x_j))/(\max(x_j)-\min(x_j))$.

Dans `scikit-learn`, elle est implémentée de manière très simimlaire à `StandardScaler` dans [`preprocessing.MinMaxScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). 

__Question :__ Reproduisez l'analyse de la section 5.1 avec cette nouvelle transformation des données. Les résultats sont-ils différents de la section 5.1 ?

#### Transformation des variables

In [ ]:
minmax_scaler = preprocessing.MinMaxScaler()
minmax_scaler.fit(X_train)

In [ ]:
X_train_minmax = minmax_scaler.transform(X_train)
X_test_minmax = minmax_scaler.transform(X_test)

#### Visualisation des variables

In [ ]:
fig = plt.figure(figsize=(8, 10))

# Histogrammes pour les variables continues
for (plot_idx, feat_idx) in enumerate(continuous_features_idx):
    # Comme précédemment
    ax = fig.add_subplot(4, 2, (2*plot_idx+1))
    h = ax.hist(X_train[:, feat_idx], bins=30, edgecolor='none')
    ax.set_title("%s (original)" % features[feat_idx])    
    
    ax = fig.add_subplot(4, 2, (2*plot_idx+2))
    h = ax.hist(X_train_scaled[:, feat_idx], bins=30, edgecolor='none', color='orange', 
                alpha=0.75 # on ajoute un peu de transparence
               )
    ax.set_title("%s (centrée-réduite)" % features[feat_idx])

    # Superposer l'histogramme pour la nouvelle transformation sur la précédente
    h = ax.hist(X_train_minmax[:, feat_idx], bins=30, edgecolor='none', color='purple', 
                alpha=0.75 # on ajoute un peu de transparence
               )
    ax.set_title("%s (minmax)" % features[feat_idx])
    
# espacement entre les subplots
fig.tight_layout(pad=1.0)

#### Impact sur le modèle

In [ ]:
# Instancier un nouvel objet LinearRegression 
predictor_minmax = linear_model.LinearRegression()

# Entraîner predictor_minmax sur les nouvelles données
predictor_minmax.fit(X_train_minmax, y_train)

In [ ]:
y_pred_minmax = predictor_minmax.predict(X_test_minmax)

In [ ]:
print(f"RMSE (scaled): {metrics.root_mean_squared_error(y_test, y_pred_minmax):.2f}")

In [ ]:
# Comme précédemment pour les coefficients de régression dans predictor
num_features = X_train.shape[1]
plt.scatter(range(num_features), np.abs(predictor.coef_), label='Originales')

# Comme précédemment pour les coefficients de régression dans predictor_scaled
plt.scatter(range(num_features), np.abs(predictor_scaled.coef_), label='Centrées-réduites', marker='v')

# Même code modifié pour afficher les coefficients de régression dans predictor_minmax
plt.scatter(range(num_features), np.abs(predictor_minmax.coef_), label='Minmax', marker='^')

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features), feature_names, rotation=90)
tmp = plt.ylabel('Coefficient')
plt.legend(loc=(0.02, 0.65))

__Interprétation :__ Toujours le même modèle prédictif, les poids changent car les fourchettes ne sont pas les mêmes mais l'importante respective des variables est intouchée.

## 6. Normalisation des variables

Vous l'aurez remarqué en regardant les histogrammes : nos variables continues ne semblent pas suivre une distribution normale. 

Dans le cas de la régression linéaire, nous n'avons fait aucune hypothèse sur la normalité des variables : nous avons supposés que les résidus sont normalement distribués. Cependant, transformer les variables pour les rapprocher de gaussiennes peut permettre d'améliorer les modèles, en particulier en contrôlant l'[asymétrie](https://fr.wikipedia.org/wiki/Asym%C3%A9trie_(statistiques)) des valeurs. 

`scikit-learn` permet d'appliquer deux types de transformations normales des variables : 
* la __transformation Box-Cox__, qui ne s'applique qu'à des variables non nulles positives. C'est cette première que nous allons illustrer ici.
* la __transformation de Yeo-Johnson__.
  
Ces deux méthodes sont disponibles dans la classe [`sklearn.preprocessing.PowerTransformer`](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-transformer) et vous pouvez en lire plus à leur sujet dans [la doc](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-transformer).

__Remarque pour aller plus loin:__ Un histogramme n'est pas un très bon moyen de vérifier qu'une distribution empirique correspond à une distribution théorique. On leur préfère plutôt un [diagramme quantile-quantile (QQ-plot)](https://fr.wikipedia.org/wiki/Diagramme_quantile-quantile), que l'on peut construire avec [la méthode `probplot` de `scipy.stats`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html) ou... un test statistique (pour la normalité, on utilise par exemple le [test de Shapiro-Wilk](https://fr.wikipedia.org/wiki/Test_de_Shapiro-Wilk) (pour se comparer à une loi normale) ou le [test de Kolmogorov-Smirnov](https://fr.wikipedia.org/wiki/Test_de_Kolmogorov-Smirnov) (pour se comparer à une loi dont on a la fonction de répartition). 

### Transformation Box-Cox des variables

In [ ]:
boxcox_scaler = preprocessing.PowerTransformer(method='box-cox')
boxcox_scaler.fit(X_train)

In [ ]:
X_train_boxcox = boxcox_scaler.transform(X_train)
X_test_boxcox = boxcox_scaler.transform(X_test)

### Visualisation des variables

__Question :__ Afficher, pour chaque variable continue, son histogramme dans `X_train_scaled`, et lui superposer (dans une autre couleur contrastante) son histogramme dans `X_train_boxcox`

In [ ]:
fig = plt.figure(figsize=(10, 6))

# Même code que précédemment pour afficher les histogrammes des variables de X_train_scaled sur une grille 2x2
for (plot_idx, feat_idx) in enumerate(continuous_features_idx): 
    ax = fig.add_subplot(2, 2, (plot_idx+1))
    h = ax.hist(X_train_scaled[:, feat_idx], bins=30, edgecolor='none', color='orange', alpha=0.75, 
               label='centrée-réduite')
    
    # superposer l'histogramme pour la nouvelle transformation 
    h = ax.hist(X_train_boxcox[:, feat_idx], bins=30, edgecolor='none', color='purple', alpha=0.75,
               label='Box-Cox')
    
    ax.set_title("%s" % features[feat_idx])   
    ax.legend(loc=(0.7, 0.7))
    
# espacement entre les subplots
fig.tight_layout(pad=1.0)

### Impact sur le modèle

__Question :__ Quel est l'impact de cette transformation sur le modèle ? 

#### Impact sur la performance

In [ ]:
# Instancier un nouvel objet LinearRegression 
predictor_boxcox = linear_model.LinearRegression()

# Entraîner predictor_boxcox sur les nouvelles données
predictor_boxcox.fit(X_train_boxcox, y_train)

In [ ]:
y_pred_boxcox = predictor_boxcox.predict(X_test_boxcox)

In [ ]:
print(f"RMSE (scaled): {metrics.root_mean_squared_error(y_test, y_pred_boxcox):.2f}")

La performance s'est améliorée !

#### Impact sur les coefficients de régression

In [ ]:
# Afficher, pour chaque variable, son poids dans le modèle (en valeur absolue)
num_features = X_train.shape[1]
plt.scatter(range(num_features), np.abs(predictor.coef_), label='Originales')
plt.scatter(range(num_features), np.abs(predictor_scaled.coef_), label='Centrées-réduites', marker='v')
plt.scatter(range(num_features), np.abs(predictor_boxcox.coef_), label='Box-Cox', marker='^')

plt.xlabel('Variable')
tmp = plt.xticks(range(num_features), feature_names, rotation=90)
tmp = plt.ylabel('Coefficient')
plt.legend(loc=(0.02, 0.65))

En utilisant la transformation Box-Cox, on donne maintenant plus d'importance à l'accélération et aux CV, et moins au poids du véhicule.

#### Comparaison aux prédictions de la baseline

Nous pouvons aussi comparer les prédictions faites par ce nouveau modèles à celles de la _baseline_ :

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_test, y_pred, label='Baseline')
plt.scatter(y_test, y_pred_boxcox, label='Box-Cox')

plt.xlabel("Consommation réelle (mpg)")
plt.ylabel("Consommation prédite (mpg)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y_test), np.min(y_pred), np.min(y_pred_boxcox)])-1
axis_max = np.max([np.max(y_test), np.max(y_pred), np.max(y_pred_boxcox)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

plt.legend(loc=(0.02, 0.8))

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(y_pred, y_pred_boxcox)

plt.xlabel("Consommation prédite (mpg) (baseline)")
plt.ylabel("Consommation prédite (mpg) (Box-Cox)")
plt.title("Régression linéaire")

# Mêmes valeurs sur les deux axes
axis_min = np.min([np.min(y_pred), np.min(y_pred_boxcox)])-1
axis_max = np.max([np.max(y_pred), np.max(y_pred_boxcox)])+1
plt.xlim(axis_min, axis_max)
plt.ylim(axis_min, axis_max)
  
# Diagonale y=x
plt.plot([axis_min, axis_max], [axis_min, axis_max], 'k-')

In [ ]:
import scipy.stats as st

In [ ]:
r, pval = st.pearsonr(y_pred, y_pred_boxcox)
print(f"Corrélation entre les prédictions : {r:.2f} (p={pval:.2e})")

__Remarque :__ Les prédictions sont très corrélées avec celles obtenues avec la deuxième transformation : le modèle est différent et sa performance semble meilleure, mais cette différence n'est peut-être pas significative.

## 7. Pour aller plus loin

Le pré-traitement des données est une partie importante du travail de _data scientist_. Voici quelques ressources et remarques pour aller plus loin :
* La transformation en vecteurs de données non-structurées (telles que texte ou images) est possible à travers des techniques telles que :
  * pour le texte : les approches bag-of-word ou tf-idf (voir [la doc scikit-learn](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) ou [le cours OpenClassrooms Analysez vos données textuelles](https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles)) ;
  * pour les images : les approches telles que SIFT (voir [le cours OpenClassrooms Classez et segmentez des données visuelles](https://openclassrooms.com/fr/courses/4470531-classez-et-segmentez-des-donnees-visuelles)).
  
* Les méthodes à noyaux et l'apprentissage profond, que nous aborderons brièvement à la fin de ce cours, permettent d'aborder autrement la représentation de données non-structurées.

* https://github.com/mirekphd/awesome-feature-engineering
* https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/